In [28]:
import os
import re
import requests
import statistics
from src.najdi_rok import najdi_rok
import pandas as pd

In [29]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [30]:
kam = 'data_raw/nobelistvo'

In [31]:
if not os.path.exists(kam):
    os.makedirs(kam)

In [32]:
koho = pd.read_json(os.path.join('data_raw','wikidata_nobelovky_pro_filtr.json'))

In [33]:
sloupce = ['100','245','700']

In [34]:
df = pd.read_parquet(os.path.join('data/cnb_sloupce','008.parquet'))
for s in sloupce:
    df = df.merge(pd.read_parquet(os.path.join('data/cnb_sloupce',f'{s}.parquet')), left_index=True, right_index=True, how='outer')

In [35]:
df.sample(10)

,008,100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,245_v,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_n,700_k,700_m,700_r,700_p,700_o,700_s,700_e,700_6,700_x,700_f,700_g
001,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bk196005367,[981202s1960 xr e ||| 0 cze],[1],"[Houška, Vojtěch,]",[jk01042489],[aut],[1915-],None,None,None,None,[1],[0],[Boj národů za mír /],None,"[Vojtěch Houška, Jiří Kettner]",None,None,None,None,None,None,[1],"[Kettner, Jiří]",[aut],None,[jk01060104],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
nkc20122341490,[120207s2011 xr ab c p 001 0 cze],[1],"[Bednaříková, Jarmila,]",[jk01011338],[aut],[1952-],None,None,None,None,[1],[0],[Dějepis :],[vzdělávací oblast Člověk a společnost.],"[Jarmila Bednaříková, Lubor Kysučan, Marie Fej...",None,[Pravěk a starověk : učebnice /],None,None,None,None,"[1, 1]","[Kysučan, Lubor,, Fejfušová, Marie]","[aut, aut]","[1968-, None]","[mzk2003181888, mzk2009512620]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
bk196401645,[971008s1964 xr cze],[1],"[Kurzweilová, Věra,]",[jk01071033],[aut],[1925-],None,None,None,None,[1],[0],[Richard Lauda :],[[Obr. publ.] /],[Věra Kurzweilová],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nkc20233548828,[230919s2022 xr abc b f 000 0 cze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Poznávací atlas :],[ilustrovaná vzdělávací kniha pro děti.],None,None,[Region Prostějovsko],None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bk195100117,[980120s1950 xr cze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Aerodynamické charakteristiky křídel.],None,None,[[Díl] 1],None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nkc20193092471,[190425s2019 xr acfhe 000 0acze],[1],"[Pollak-Kinsky, Helga,]",[zmp2014818034],[aut],[1930-2020],None,None,None,None,[1],[0],[Můj terezínský deník 1943-1944 a zápisky mého...,None,"[Helga Pollak-Kinsky ; k vydání připravila, ko...",None,None,None,None,None,None,"[1, 1, 1]","[Pollak, Otto,, Brenner-Wonschick, Hannelore,,...","[aut, cwt, trl]","[1894-1978, 1951-, 1986-]","[mzk2016924663, jx20041216001, xx0191304]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
nkc20051326307,[040614d20012009xr ar ff 0 b0cze],[1],"[Bendl, Karel]",[xx0005710],[aut],None,None,None,None,None,[1],[0],[Události třetího tisíciletí /],None,[Karel Bendl a kolektiv],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk20112174639,[010907s1879 xr f ger d],[1],"[Elvert, Christian d',]",[jk01030444],[aut],[1803-1896],None,None,None,None,[1],[0],[Zur Geschichte der Volks-Schulen in Brünn /],None,[Chr. d'Elvert],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpg20112267814,[111228s2011 xr nnn g kncze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Jazz Time],[9.11.[2011] Praha 1 : Ancient & Mekons Modern],None,None,None,[[grafika] :],None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
for index, row in koho.iterrows():
    nazev_souboru = f"nobelovka_{row['nobelovka']}_{row['100_7']}.parquet"
    if nazev_souboru not in os.listdir(kam):
        print(row['nobelovka'])
        df_slice = df[df['100_7'].astype(str).str.contains(row['100_7'], na=False) | df['700_7'].astype(str).str.contains(row['100_7'], na=False)]
        df_slice = df_slice.explode('008')
        df_slice['rok'] = df_slice['008'].apply(lambda x: najdi_rok(x))
        df_slice['nobelovka'] = row['nobelovka']
        df_slice['titul'] = df_slice['245_a'].apply(lambda x: x[0].replace(' /','').replace('=','').replace(' :','').replace(';','').strip() if x != None else None)
        df_slice.to_parquet(os.path.join(kam, nazev_souboru))

1901
